In [1]:
!pip install nltk pymorphy2 umap-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.0 MB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 27.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 59.8 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 62.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 MB 45.5 MB/s eta 0:00:0000:0100:01
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=f7ba7217020385fa97f4959edd224e90c14f5c5c83fdcde90ab6f03b7f7b69fb
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none

In [31]:
!mkdir vectorizers

In [24]:
import umap
import pandas as pd
#import mlflow
from sklearn.model_selection import train_test_split, ParameterGrid
#import torch
#from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from tqdm.notebook import tqdm
import pymorphy2
import pandas as pd
morph = pymorphy2.MorphAnalyzer()
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords');
stopwords = set(stopwords.words('russian'))
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
train = pd.read_csv("kimbris/data/train_with_embeddings.csv")
test = pd.read_csv("kimbris/data/test_with_embeddings.csv")


In [26]:
train

,Исполнитель,Группа тем,Текст инцидента,Тема,0,1,2,3,4,5,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,Город Пермь,Погребение и похоронное дело,Погребения - это серьезная проблема в нашей ст...,Погребение и похоронное дело,-0.294947,-0.055111,-0.917189,0.076128,-0.168686,0.458902,...,0.023235,1.380208,0.494589,1.078209,0.454874,0.191731,0.399481,-0.026228,0.799322,0.877953
1,Город Пермь,Погребение и похоронное дело,"Я также считаю, что мы должны делать все возмо...",Погребение и похоронное дело,-0.215702,0.045419,-1.404945,-0.113390,-0.199697,0.628931,...,0.064474,1.044024,0.080316,1.508044,0.344166,0.062843,-0.097315,0.063320,0.440753,0.565733
2,Город Пермь,Погребение и похоронное дело,"Я думаю, что многие люди уже готовы переходить...",Погребение и похоронное дело,-0.074277,-0.069510,-1.307976,-0.731061,-0.117746,0.112123,...,-0.352016,1.426871,0.228109,1.613876,-0.120798,0.152127,0.561417,0.173413,0.980470,0.895009
3,Город Пермь,Погребение и похоронное дело,"Я считаю, что строительство новых крематориев ...",Погребение и похоронное дело,-0.215675,-0.010250,-1.629285,-0.177629,-0.034335,0.769009,...,0.172642,1.292364,0.157263,1.586579,0.690120,-0.019305,0.203964,-0.059195,0.468472,0.829206
4,Город Пермь,Погребение и похоронное дело,"Я думаю, что мы должны обсуждать этот вопрос о...",Погребение и похоронное дело,0.091469,-0.337995,-1.098347,-0.416570,0.188040,0.489746,...,0.567005,2.010411,-0.007483,2.252533,1.322259,0.245822,-0.209317,0.417235,0.323693,0.814260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16847,ИГЖН ПК,ЖКХ,"Счёт за отопление пришёл за 1,5 месяца октябр...",Завышение платы за коммунальные услуги,0.494950,-0.152662,-1.038108,-0.324845,0.389979,0.587082,...,0.546728,0.547754,0.729002,0.792730,0.791029,0.062633,0.046741,0.662049,-0.931667,0.543606
16848,Министерство социального развития ПК,Социальное обслуживание и защита,"Здравствуйте, справку о том что я не получала...",Оказание гос. соц. помощи,-0.245126,-0.062770,-0.160016,0.977306,-0.091960,1.371442,...,0.213309,0.901905,0.367159,0.690490,0.633698,0.277229,-0.180435,-0.357861,-0.234448,0.931398
16849,ИГЖН ПК,ЖКХ,"Добрый день, куда и к кому обращаться о ненад...",Ненадлежащее содержание и эксплуатация МКД,0.096977,0.334582,0.256389,0.675268,-0.216912,0.327004,...,0.555713,0.927510,0.054273,-0.680263,-0.086192,-0.246548,0.210988,0.341651,-0.130749,0.731547
16850,ИГЖН ПК,ЖКХ,"Приморский 19, управляйка не чистит снег с кр...",★ Наледь и сосульки на кровле,-0.198398,-0.084078,-0.520404,-1.223011,0.486935,0.555129,...,0.514814,0.568372,0.385218,0.999179,-0.845614,-0.205630,0.443906,0.574016,-0.982858,0.722479


In [27]:
def lemmatize(text):
    words = text.split() # разбиваем текст на слова
    #print(words)
    res = list()
    for word in words:
        if (word not in stopwords and len(word) > 1):
            p = morph.parse(word)[0]
            res.append(p.normal_form)
    text = " ".join(res)
    return text

def text_cleaner(input_text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', input_text)
    rem_url = re.sub(r'http\S+', '',cleantext)
    rem_url= re.sub(r'([a-z]{2}\d+[a-z]{2})',' ',rem_url)
    rem_url = re.sub(r'!+','.', rem_url)
    rem_num = re.sub('[0-9]+', '', rem_url)
    text = re.sub(r"[-—()\"#/@;:<>{}=~|€«»$\+'_–\*°“”\\√&×•ó÷≈„()‽\+.,!]+", " ", rem_num)
    text = re.sub("!",".", text)
    text = re.sub(r' +', ' ', text)
    text = re.sub(r"\[.+\]",'', text)
    emoji_pattern=re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

    text = emoji_pattern.sub('', text)
    text = text.lower()
    return text


def dataset_preprocessor_vectorizers(data, w_count_to_drop=3):
    data_frame = data.copy()

    indexes_to_drop = []
    # Дропаем дубли
    #data_frame = data_frame.drop_duplicates(subset=['Текст инцидента']).reset_index(drop=True)

    # Regex для групп
    p = re.compile("\[.+\|.+\]")

    for index, row in tqdm(data_frame.iterrows()):

        # Очистка текста
        data_frame.loc[index, "Текст инцидента"] = text_cleaner(data_frame.loc[index, "Текст инцидента"])

        # Отсекаем сообщения из w_count_to_drop слов
        if len(row["Текст инцидента"].split(" ")) < w_count_to_drop and len(row["Текст инцидента"].split(".")) < w_count_to_drop:
            indexes_to_drop.append(index)

    # Дропаем индексы < w_count_to_drop слов
    #print(indexes_to_drop)
    #data_frame = data_frame.drop(index=indexes_to_drop).reset_index(drop=True)

    # Лемматизация и удаление стоп слов
    for index, row in tqdm(data_frame.iterrows()):
        data_frame.loc[index, "Текст инцидента"] = lemmatize(row["Текст инцидента"])
    return data_frame
train = dataset_preprocessor_vectorizers(train)
test = dataset_preprocessor_vectorizers(test)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [28]:
train

,Исполнитель,Группа тем,Текст инцидента,Тема,0,1,2,3,4,5,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,Город Пермь,Погребение и похоронное дело,погребение это серьёзный проблема наш страна д...,Погребение и похоронное дело,-0.294947,-0.055111,-0.917189,0.076128,-0.168686,0.458902,...,0.023235,1.380208,0.494589,1.078209,0.454874,0.191731,0.399481,-0.026228,0.799322,0.877953
1,Город Пермь,Погребение и похоронное дело,также считать должный делать возможный поддерж...,Погребение и похоронное дело,-0.215702,0.045419,-1.404945,-0.113390,-0.199697,0.628931,...,0.064474,1.044024,0.080316,1.508044,0.344166,0.062843,-0.097315,0.063320,0.440753,0.565733
2,Город Пермь,Погребение и похоронное дело,думать многие человек готовый переходить альте...,Погребение и похоронное дело,-0.074277,-0.069510,-1.307976,-0.731061,-0.117746,0.112123,...,-0.352016,1.426871,0.228109,1.613876,-0.120798,0.152127,0.561417,0.173413,0.980470,0.895009
3,Город Пермь,Погребение и похоронное дело,считать строительство новый крематорий стать в...,Погребение и похоронное дело,-0.215675,-0.010250,-1.629285,-0.177629,-0.034335,0.769009,...,0.172642,1.292364,0.157263,1.586579,0.690120,-0.019305,0.203964,-0.059195,0.468472,0.829206
4,Город Пермь,Погребение и похоронное дело,думать должный обсуждать вопрос открыто честно...,Погребение и похоронное дело,0.091469,-0.337995,-1.098347,-0.416570,0.188040,0.489746,...,0.567005,2.010411,-0.007483,2.252533,1.322259,0.245822,-0.209317,0.417235,0.323693,0.814260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16847,ИГЖН ПК,ЖКХ,счёт отопление прийти месяц октябрь пол сентяб...,Завышение платы за коммунальные услуги,0.494950,-0.152662,-1.038108,-0.324845,0.389979,0.587082,...,0.546728,0.547754,0.729002,0.792730,0.791029,0.062633,0.046741,0.662049,-0.931667,0.543606
16848,Министерство социального развития ПК,Социальное обслуживание и защита,здравствуйте справка получать пособие рождение...,Оказание гос. соц. помощи,-0.245126,-0.062770,-0.160016,0.977306,-0.091960,1.371442,...,0.213309,0.901905,0.367159,0.690490,0.633698,0.277229,-0.180435,-0.357861,-0.234448,0.931398
16849,ИГЖН ПК,ЖКХ,добрый день кто обращаться ненадлежащий состоя...,Ненадлежащее содержание и эксплуатация МКД,0.096977,0.334582,0.256389,0.675268,-0.216912,0.327004,...,0.555713,0.927510,0.054273,-0.680263,-0.086192,-0.246548,0.210988,0.341651,-0.130749,0.731547
16850,ИГЖН ПК,ЖКХ,приморский управляйка чистить снег крыша вчера...,★ Наледь и сосульки на кровле,-0.198398,-0.084078,-0.520404,-1.223011,0.486935,0.555129,...,0.514814,0.568372,0.385218,0.999179,-0.845614,-0.205630,0.443906,0.574016,-0.982858,0.722479


In [29]:
import pickle
class Model_tfidf_emb_umap:
    def __init__(self, vectorizer_name, vectorizer, max_features=None, emb_length=1024, emb_umap_size=100, vectorizer_umap_size=500):
        self.max_features = max_features
        self.vectorizer = vectorizer(max_features=self.max_features)
        self.vectorizer_name = vectorizer_name
        self.emb_length = emb_length
        self.emb_umap_size = emb_umap_size
        self.vectorizer_umap_size = vectorizer_umap_size
        
    def create_umap(self, emb_names, train, test, cols_for_umap, n_components=100):
        column_names = [emb_names+f"_{i}" for i in range(n_components)]
        ump = umap.UMAP(n_components=n_components, random_state=42)
        train_data_frame = train.copy()
        test_data_frame = test.copy()
        train_emb = ump.fit_transform(train_data_frame[cols_for_umap])
        test_emb = ump.transform(test_data_frame[cols_for_umap])
        
        return pd.DataFrame(train_emb, columns=column_names) , pd.DataFrame(test_emb, columns=column_names)
    
    def create_tfidf(self, train, test):
      tf = self.vectorizer.fit_transform(train)
      with open(f'vectorizers/{self.vectorizer_name}_vectorizer_{self.max_features}.pk', 'wb') as fin:
        pickle.dump(self.vectorizer, fin)
      vocab = self.vectorizer.vocabulary_
      train_tf, test_tf = self.create_umap("tfidf", pd.DataFrame(tf.toarray(), columns=sorted(vocab.keys())),
                             pd.DataFrame(self.vectorizer.transform(test).toarray(), columns=sorted(vocab.keys())),
                                           cols_for_umap=sorted(vocab.keys()), 
                                           n_components=self.vectorizer_umap_size)
      del tf
      return train_tf, test_tf


    def create_datasets(self, train, test):
        train_, test_ = train.copy(), test.copy()
        
        embeddings_columns = [str(i) for i in range(self.emb_length)]
        train_emb, test_emb = self.create_umap("bert_emb", 
                                          train_[embeddings_columns] ,
                                          test_[embeddings_columns], 
                                          embeddings_columns, self.emb_umap_size)
        train_ = train_[["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]]
        test_ = test_[["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]]
        
        train_ = pd.concat([train_, train_emb], axis=1)
        test_ = pd.concat([test_, test_emb], axis=1)
        
        train_tf_idf, test_tf_idf = self.create_tfidf(train_["Текст инцидента"].to_list(), test_["Текст инцидента"].to_list())
        
       
        
        train_ = pd.concat([train_, train_tf_idf], axis=1)
        test_ = pd.concat([test_, test_tf_idf], axis=1)
        
        del train_tf_idf
        del test_tf_idf
        #train_.to_csv(f"datasets/{self.vectorizer_name}_train_{self.max_features}.csv", index=False)
        #test_.to_csv(f"datasets/{self.vectorizer_name}_test_{self.max_features}.csv", index=False)

        return train_, test_

In [32]:
mtf = Model_tfidf_emb_umap("tfidf", TfidfVectorizer, max_features=15000, emb_length=1024, emb_umap_size=100, vectorizer_umap_size=500) 
train_, test_ = mtf.create_datasets(train, test)

In [33]:
columns = ["bert_emb"+f"_{i}" for i in range(100)] + ["tfidf"+f"_{i}" for i in range(500)]
new_columns = [i for i in range(600)]
dictionary = dict(zip(columns, new_columns))
train_ = train_.rename(dictionary, axis='columns')
test_ = test_.rename(dictionary, axis='columns')

In [34]:
test_

,Текст инцидента,Группа тем,Исполнитель,Тема,0,1,2,3,4,5,...,590,591,592,593,594,595,596,597,598,599
0,подсказать номер телефон обратиться жалоба пож...,Здравоохранение/Медицина,Министерство здравоохранения,Хамство медицинских работников,2.089460,4.257178,9.228863,3.228595,4.063317,2.834913,...,5.770841,4.270641,3.388572,5.445230,4.085342,8.797091,4.926394,7.753830,4.539584,5.226314
1,здравствуйте открыться бассейн снова?,Физическая культура и спорт,Лысьвенский городской округ,Ремонт спортивных учреждений,4.843365,3.425531,9.109115,2.863883,3.889273,2.688141,...,5.769928,4.270021,3.393765,5.440283,4.080599,8.801817,4.931379,7.748927,4.527998,5.235141
2,житель хотеть попросить глава город глава райо...,Дороги,Город Пермь,★ Нарушение правил очистки дорог от снега и на...,7.001543,4.005813,10.122937,2.465279,5.025868,1.921851,...,5.766040,4.276530,3.386585,5.443857,4.089214,8.791391,4.926282,7.765127,4.544145,5.236749
3,фап строить фельдшер наслать который согласный...,Здравоохранение/Медицина,Министерство здравоохранения,Нехватка или сокращение врачей и медицинских у...,1.564634,5.980015,9.694421,3.116809,5.501141,2.520229,...,5.768358,4.271991,3.395011,5.443560,4.081778,8.805858,4.928887,7.750834,4.527453,5.234826
4,власов каждый зима кошмар твориться подъехать ...,Благоустройство,ИГЖН ПК,★ Нарушение правил уборки от снега и наледи вн...,8.276342,4.277385,11.275164,2.640700,4.791358,1.997465,...,5.772738,4.269926,3.393125,5.437267,4.088700,8.799204,4.930578,7.752034,4.530592,5.235062
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7430,здравствуйте срок исполнение справка малоимущн...,Социальное обслуживание и защита,Министерство социального развития ПК,Оказание гос. соц. помощи,1.985291,3.039652,10.919013,3.453451,3.661172,3.024125,...,5.774777,4.269431,3.384973,5.444088,4.088180,8.813890,4.921966,7.757819,4.541751,5.217953
7431,хлебозаводский такой ещё плохой,Дороги,Город Пермь,Ямы и выбоины на дороге,7.640713,4.036338,10.881562,2.848541,5.156816,2.208190,...,5.772928,4.269828,3.385810,5.443447,4.088383,8.815413,4.916113,7.756968,4.540256,5.225392
7432,это забота пациентах? послать диспансеризация ...,Здравоохранение/Медицина,Министерство здравоохранения,★ Оказание медицинской помощи не в полном объе...,0.679040,6.352221,9.767119,3.267550,5.193417,2.694814,...,5.772035,4.269053,3.391553,5.442033,4.081958,8.807399,4.928972,7.751571,4.529467,5.240129
7433,проблема оптимизация сфера здравоохранение наш...,Здравоохранение/Медицина,Министерство здравоохранения,Нехватка материально-технического обеспечения,1.262043,5.610096,9.746059,3.214051,5.107607,2.661624,...,5.774309,4.268971,3.393668,5.442321,4.082699,8.800743,4.921329,7.763759,4.531266,5.237947


In [35]:
train_.to_csv("kimbris/data/umap_data/train_with_embeddings_tfidf_umap_rename.csv", index=False)
test_.to_csv("kimbris/data/umap_data/test_with_embeddings_tfidf_umap_rename.csv", index=False)

In [ ]:
train = pd.read_csv("data/umap_data/train_with_embeddings_tfidf_umap_new_rename.csv") 
test = pd.read_csv("data/umap_data/test_with_embeddings_tfidf_umap_new_rename.csv") 

In [11]:
param_grid = {
    'iterations': [100, 250, 400, 500]
}
grid = list(ParameterGrid(param_grid))

In [12]:
mlflow.set_experiment('generated_sbert100_tfidf500_umap_new')

2023/11/25 00:26:29 INFO mlflow.tracking.fluent: Experiment with name 'generated_sbert100_tfidf500_umap' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///workspace/mlruns/990915311576114456', creation_time=1700871989743, experiment_id='990915311576114456', last_update_time=1700871989743, lifecycle_stage='active', name='generated_sbert100_tfidf500_umap', tags={}>

In [14]:
#for model_ in models:
    #model_name, vectorizer, max_features = model_
    #model = Model_tfidf(model_name, vectorizer, max_features)



for params in grid:
    try:
        with mlflow.start_run(nested=True):
            catboost_model = CatBoostClassifier(**params, verbose=0, random_seed=42)
            catboost_model.fit(train_.drop(columns=["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]), train_["Группа тем"])

            predictions = catboost_model.predict(test_.drop(columns=["Текст инцидента", "Группа тем", "Исполнитель", "Тема"]))

            accuracy = accuracy_score(test_["Группа тем"], predictions)
            f1 = f1_score(test_["Группа тем"], predictions, average='weighted')
            report = classification_report(test_["Группа тем"], predictions, output_dict=True)
            report_text =  classification_report(test_["Группа тем"], predictions)


            mlflow.log_metric("report_accuracy", report['accuracy'])
            mlflow.log_metric("macro avg_precision", report['macro avg']['precision'])
            mlflow.log_metric("macro avg_recall", report['macro avg']['recall'])
            mlflow.log_metric("macro avg_f1-score", report['macro avg']['f1-score'])
            mlflow.log_metric("weighted avg_precision", report['weighted avg']['precision'])
            mlflow.log_metric("weighted avg_recall", report['weighted avg']['recall'])
            mlflow.log_metric("weighted avg_f1-score", report['weighted avg']['f1-score'])

            mlflow.log_text(report_text, "classification_report.txt")

            mlflow.set_tag("umap_emb_size", 100)
            mlflow.set_tag("umap_tfidf_size", 500)
            mlflow.set_tag("max_features", 15000)
            mlflow.set_tag("dataset_name", "sbert100_tfidf350_umap")
            mlflow.set_tag("model_name", "catboost")
            mlflow.catboost.log_model(catboost_model, "model")
            mlflow.log_params(params)
            mlflow.log_metrics({'accuracy': accuracy, 'f1-weighted': f1})
    except Exception as e:
        error_name = type(e).name
        print(f"Caught an error: {error_name}")
        print(f"EXCEPTION: {e}")
        print(sys.exc_info()[0])
        traceback.print_exc(file=sys.stdout)

In [17]:
train_

,Текст инцидента,Группа тем,Исполнитель,Тема,bert_emb_0,bert_emb_1,bert_emb_2,bert_emb_3,bert_emb_4,bert_emb_5,...,tfidf_490,tfidf_491,tfidf_492,tfidf_493,tfidf_494,tfidf_495,tfidf_496,tfidf_497,tfidf_498,tfidf_499
0,погребение это серьёзный проблема наш страна д...,Погребение и похоронное дело,Город Пермь,Погребение и похоронное дело,6.193858,3.912866,8.221570,2.597436,2.557736,5.019467,...,4.492201,3.140940,4.713594,4.526647,5.208543,4.716970,4.446305,5.084937,2.404349,3.693383
1,также считать должный делать возможный поддерж...,Погребение и похоронное дело,Город Пермь,Погребение и похоронное дело,6.182034,3.913239,8.219021,2.584416,2.539699,5.037027,...,4.491259,3.140860,4.718349,4.532989,5.213480,4.726561,4.450813,5.087780,2.409021,3.698841
2,думать многие человек готовый переходить альте...,Погребение и похоронное дело,Город Пермь,Погребение и похоронное дело,6.193626,3.921363,8.194581,2.599775,2.571837,5.004468,...,4.493160,3.140112,4.713707,4.523725,5.206819,4.712420,4.443615,5.085279,2.399018,3.691391
3,считать строительство новый крематорий стать в...,Погребение и похоронное дело,Город Пермь,Погребение и похоронное дело,6.166926,3.913481,8.211468,2.594467,2.546340,5.036290,...,4.492138,3.141362,4.713727,4.526781,5.208986,4.717175,4.446662,5.084739,2.404690,3.693920
4,думать должный обсуждать вопрос открыто честно...,Погребение и похоронное дело,Город Пермь,Погребение и похоронное дело,5.426589,4.137989,7.872371,2.600991,2.342771,5.481013,...,4.489232,3.136314,4.702068,4.518471,5.203340,4.704942,4.440556,5.078983,2.408298,3.685203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16552,счёт отопление прийти месяц октябрь пол сентяб...,ЖКХ,ИГЖН ПК,Завышение платы за коммунальные услуги,5.802146,3.359223,11.347735,3.234591,1.857580,6.685138,...,4.465665,3.113088,4.680392,4.528454,5.215415,4.736930,4.474161,5.057438,2.467376,3.693809
16553,здравствуйте справка получать пособие рождение...,Социальное обслуживание и защита,Министерство социального развития ПК,Оказание гос. соц. помощи,1.978383,2.243957,10.591244,3.414699,1.842726,6.676864,...,4.459016,3.087916,4.657785,4.513838,5.203963,4.716463,4.460990,5.038879,2.480016,3.673120
16554,добрый день обращаться ненадлежащий состояние ...,ЖКХ,ИГЖН ПК,Ненадлежащее содержание и эксплуатация МКД,5.127955,3.364052,10.492113,2.981782,1.837409,6.203472,...,4.478580,3.157574,4.708783,4.550032,5.213682,4.758401,4.464355,5.088152,2.442261,3.697817
16555,приморский управляйка чистить снег крыша вчера...,ЖКХ,ИГЖН ПК,★ Наледь и сосульки на кровле,7.862041,4.475359,11.675790,2.809377,1.940344,5.739951,...,4.475419,3.171743,4.701551,4.547494,5.213686,4.759737,4.473089,5.074672,2.447324,3.700868


In [22]:
columns = ["bert_emb"+f"_{i}" for i in range(100)] + ["tfidf"+f"_{i}" for i in range(500)]
new_columns = [i for i in range(600)]
dictionary = dict(zip(columns, new_columns))
train_ = train_.rename(dictionary, axis='columns')
test_ = test_.rename(dictionary, axis='columns')

,Текст инцидента,Группа тем,Исполнитель,Тема,0,1,2,3,4,5,...,590,591,592,593,594,595,596,597,598,599
0,погребение это серьёзный проблема наш страна д...,Погребение и похоронное дело,Город Пермь,Погребение и похоронное дело,6.193858,3.912866,8.221570,2.597436,2.557736,5.019467,...,4.492201,3.140940,4.713594,4.526647,5.208543,4.716970,4.446305,5.084937,2.404349,3.693383
1,также считать должный делать возможный поддерж...,Погребение и похоронное дело,Город Пермь,Погребение и похоронное дело,6.182034,3.913239,8.219021,2.584416,2.539699,5.037027,...,4.491259,3.140860,4.718349,4.532989,5.213480,4.726561,4.450813,5.087780,2.409021,3.698841
2,думать многие человек готовый переходить альте...,Погребение и похоронное дело,Город Пермь,Погребение и похоронное дело,6.193626,3.921363,8.194581,2.599775,2.571837,5.004468,...,4.493160,3.140112,4.713707,4.523725,5.206819,4.712420,4.443615,5.085279,2.399018,3.691391
3,считать строительство новый крематорий стать в...,Погребение и похоронное дело,Город Пермь,Погребение и похоронное дело,6.166926,3.913481,8.211468,2.594467,2.546340,5.036290,...,4.492138,3.141362,4.713727,4.526781,5.208986,4.717175,4.446662,5.084739,2.404690,3.693920
4,думать должный обсуждать вопрос открыто честно...,Погребение и похоронное дело,Город Пермь,Погребение и похоронное дело,5.426589,4.137989,7.872371,2.600991,2.342771,5.481013,...,4.489232,3.136314,4.702068,4.518471,5.203340,4.704942,4.440556,5.078983,2.408298,3.685203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16552,счёт отопление прийти месяц октябрь пол сентяб...,ЖКХ,ИГЖН ПК,Завышение платы за коммунальные услуги,5.802146,3.359223,11.347735,3.234591,1.857580,6.685138,...,4.465665,3.113088,4.680392,4.528454,5.215415,4.736930,4.474161,5.057438,2.467376,3.693809
16553,здравствуйте справка получать пособие рождение...,Социальное обслуживание и защита,Министерство социального развития ПК,Оказание гос. соц. помощи,1.978383,2.243957,10.591244,3.414699,1.842726,6.676864,...,4.459016,3.087916,4.657785,4.513838,5.203963,4.716463,4.460990,5.038879,2.480016,3.673120
16554,добрый день обращаться ненадлежащий состояние ...,ЖКХ,ИГЖН ПК,Ненадлежащее содержание и эксплуатация МКД,5.127955,3.364052,10.492113,2.981782,1.837409,6.203472,...,4.478580,3.157574,4.708783,4.550032,5.213682,4.758401,4.464355,5.088152,2.442261,3.697817
16555,приморский управляйка чистить снег крыша вчера...,ЖКХ,ИГЖН ПК,★ Наледь и сосульки на кровле,7.862041,4.475359,11.675790,2.809377,1.940344,5.739951,...,4.475419,3.171743,4.701551,4.547494,5.213686,4.759737,4.473089,5.074672,2.447324,3.700868


In [20]:
dictionary

{'bert_emb_0': 0,
 'bert_emb_1': 1,
 'bert_emb_2': 2,
 'bert_emb_3': 3,
 'bert_emb_4': 4,
 'bert_emb_5': 5,
 'bert_emb_6': 6,
 'bert_emb_7': 7,
 'bert_emb_8': 8,
 'bert_emb_9': 9,
 'bert_emb_10': 10,
 'bert_emb_11': 11,
 'bert_emb_12': 12,
 'bert_emb_13': 13,
 'bert_emb_14': 14,
 'bert_emb_15': 15,
 'bert_emb_16': 16,
 'bert_emb_17': 17,
 'bert_emb_18': 18,
 'bert_emb_19': 19,
 'bert_emb_20': 20,
 'bert_emb_21': 21,
 'bert_emb_22': 22,
 'bert_emb_23': 23,
 'bert_emb_24': 24,
 'bert_emb_25': 25,
 'bert_emb_26': 26,
 'bert_emb_27': 27,
 'bert_emb_28': 28,
 'bert_emb_29': 29,
 'bert_emb_30': 30,
 'bert_emb_31': 31,
 'bert_emb_32': 32,
 'bert_emb_33': 33,
 'bert_emb_34': 34,
 'bert_emb_35': 35,
 'bert_emb_36': 36,
 'bert_emb_37': 37,
 'bert_emb_38': 38,
 'bert_emb_39': 39,
 'bert_emb_40': 40,
 'bert_emb_41': 41,
 'bert_emb_42': 42,
 'bert_emb_43': 43,
 'bert_emb_44': 44,
 'bert_emb_45': 45,
 'bert_emb_46': 46,
 'bert_emb_47': 47,
 'bert_emb_48': 48,
 'bert_emb_49': 49,
 'bert_emb_50': 50,
